In [4]:
import glob #For locating files with a specific extension
import os #For moving between directories
import zipfile #For unzipping the input folder
from time import sleep #For timing the process of unziping
from tqdm import tqdm #For monitoring the process of unziping
import json #For loading json files
import codecs #For transforming date format
import html #For loading html files
import re #Regex
from bs4 import BeautifulSoup #For parsing the html files
import pandas as pd #Working with dataframes
import csv #For working with csv files
from pandas import json_normalize #For displaying json files better
import numpy as np
import sys 
import collections
import mailbox
import bs4
import email as e
import imaplib
import email
import getpass
import logging
import string
import datetime
from dateutil.parser import parse
import random
import email.utils
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
import lxml.html
from nltk.stem import WordNetLemmatizer, PorterStemmer
import unicodedata

# Extraction

### Zip extraction

In [19]:
init = '/mnt/c/Ironhack/Material/Repositorio/Final/data/input'

def zip_format():
    #Let's change the working directory
    os.chdir(init)
    #Let's move to the working directory.
    os.getcwd()
    #Let's check everything here with a .zip extension
    zip_list = glob.glob("*.zip")
    if len(zip_list) == 1:
        #Let's use list comprehension to extract the name of the file as a string
        zip_name = ''.join([str(item) for item in zip_list])
        zip_location = f'{init}/{zip_name}'
        return zip_location
    else:
        print('Too many .zip files detected OR there is no .zip at all')

In [20]:
def zip_extractor(*args, **kwargs):
    with zipfile.ZipFile(zip_format()) as zf:
        for member in tqdm(zf.infolist(), desc='Extracting '):
            try:
                zf.extract(member)
            except zipfile.error as e:
                pass
        zf.close()
        #os.system(f"rm -rf {zip_format()}")

In [21]:
def executor(*args, **kwarg):
    try:
        zip_extractor()
        print('Success!')
    except zipfile.error:
        raise ("It seems that there was an error during the extraction.")

In [8]:
executor()

Extracting : 100%|██████████| 2473/2473 [04:22<00:00,  9.41it/s] 

Success!


In [22]:
os.chdir(init)
temp = os.listdir()
temp1 = ''.join([str(item) for item in temp if zipfile.is_zipfile(item) == False])
path_string = f"{init}/{temp1}"
os.chdir(path_string)

In [23]:
path_string

'/mnt/c/Ironhack/Material/Repositorio/Final/data/input/Takeout'

In [24]:
os.listdir()

['Actividad de registro de accesos',
 'Aplicación Home',
 'archive_browser.html',
 'Calendar',
 'Chrome',
 'Contactos',
 'Correo',
 'Cuenta de Google',
 'Developer Platform',
 'Drive',
 'Google Fotos',
 'Google Play Películas',
 'Google Play Store',
 'Google Shopping',
 'Google Workspace Marketplace',
 'Google\xa0My\xa0Business',
 'Google\xa0Pay',
 'Hangouts',
 'Historial de ubicaciones',
 'Listas de la compra',
 'Maps',
 'Maps (Tus sitios)',
 'Mi actividad',
 'My\xa0Maps',
 'Notas y listas del Asistente',
 'Noticias',
 'Perfil',
 'Search Contributions',
 'Servicio de configuración de dispositivo Android',
 'Tareas',
 'YouTube y YouTube Music']

In [25]:
files_list = os.listdir()

In [ ]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### Cuenta de Google / (html) [***COMPLETED***]

In [13]:
def account_html():
    li = []
    tr = []
    url = f"{path_string}/Cuenta de Google"
    os.chdir(url)
    directory = os.getcwd()
    for filename in os.listdir(directory):
        if filename.endswith('.html'):
            fname = os.path.join(directory, filename)
            with open(fname, 'r') as file:
                beautifulSoupText = BeautifulSoup(file.read(), 'html.parser')
                for tag in beautifulSoupText.findAll('li'):
                    li.append(tag)
                
                for tag in beautifulSoupText.findAll('tr'):
                    tr.append(tag)
    return li, tr

def account_pd(*args, **kwargs):
    li = pd.DataFrame(data=(account_html()[0]), columns=['category'])
    return li

def log_activity_pd(*args, **kwargs):
    tr = pd.DataFrame(data=(account_html()[1]), columns = ['timestamp', 'ip', 'activity', 'agent'])
    tr.drop(['agent'], axis=1, inplace=True)
    return tr.iloc[1: , :]
    

In [14]:
li = account_pd()
tr = log_activity_pd()

/home/krujo/Anaconda3/lib/python3.8/site-packages/pandas/core/internals/construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [15]:
li.category = li.category.str.split(':')
li_final = li.category.apply(pd.Series).T
li_final.rename(columns={0:'Google Account ID',2:'Created on',3:'IP',4:'Language',5:'e-mail',6:'recovery mail',7:'phone'}, inplace=True)
li_final.drop([0, 2, 3], axis=0, inplace=True)
li_final.drop([1], axis=1, inplace=True)
li_final.reset_index(drop=True, inplace=True)

In [16]:
def ip_(x):
    return x.text

In [17]:
tr['date'] = tr.timestamp.apply(ip_)
tr['ip'] = tr.ip.apply(ip_)
tr['activity'] = tr.activity.apply(ip_)
tr.drop(columns=['timestamp'], inplace=True)

In [18]:
def get_numbers_only(x):
    return re.sub('[^0-9]', '', x)
    
li_final['phone'] = li_final['phone'].apply(lambda x: get_numbers_only(x))
li_final['n_phone'] = li_final['phone'][-9:]

In [19]:
li_final.head(2)

,Google Account ID,Created on,IP,Language,e-mail,recovery mail,phone,n_phone
0,907204290308,2017-07-20 08,213.4.207.110,es,JRDiazOrdaz03@gmail.com,IronhideX1@gmail.com,34615686348,34615686348


In [20]:
tr.head(2)

,ip,activity,date
1,2.136.118.161,Login,2021-12-06 10:27:15 UTC
2,85.48.184.103,Login,2021-12-06 10:27:11 UTC


In [ ]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### Perfil / Profile (json) [***COMPLETED***]

In [21]:
def profile_extr():
    for i in files_list:
        if i == 'Perfil':
            local = f"{path_string}/{i}/Perfil.json"
    try:
        f = open(local)
        data = json.load(f)
        name = data['name']['formattedName']
        email = (data['emails'][0]['value']).lower()
        gender = data['gender']['type']
        profile_dict = {'name': name, 'email': email, 'gender': gender}
        return profile_dict
    
    except: NameError


In [22]:
profile_extr = profile_extr()

In [23]:
df_profile_extr = pd.DataFrame(profile_extr, index=(0,))

In [24]:
df_profile_extr.head()

,name,email,gender
0,Jose Diaz,jrdiazordaz03@gmail.com,male


In [ ]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### Actividad de registro de accesos / (csv) [***COMPLETED***]

In [92]:
def activ_services():
    for i in files_list:
        if i == 'Actividad de registro de accesos':
            local = f"{path_string}/{i}/Actividades_ una lista con los servicios de Google.csv"
    df = pd.read_csv(local)
    return df

In [93]:
activ_services = activ_services()

In [94]:
activ_services.drop(columns=['Proxiedhost IP Address','Is Non-routable IP Address','Referer Product Name','Referer Sub-Product Name','Activity Type','Gmail Access Channel','Android Webview Package Name', 'User Agent String'], inplace=True)

In [28]:
activ_services.head()

,Gaia ID,Activity Timestamp,IP Address,Activity Country,Activity Region,Activity City,User Agent String,Product Name,Sub-Product Name
0,907204290308,2021-12-05 07:57:48 UTC,89.130.89.27,es,community of madrid,alcobendas,Android-Gmail/62948716 (sw360dp; 320dpi) (H331...,Gmail,Other
1,907204290308,2021-12-05 07:57:47 UTC,89.130.89.27,es,community of madrid,alcobendas,Android-Gmail/62948716 (sw360dp; 320dpi) (H331...,Gmail,Other
2,907204290308,2021-12-05 07:57:17 UTC,89.130.89.27,es,community of madrid,alcobendas,Android-Gmail/62948716 (sw360dp; 320dpi) (H331...,Gmail,Other
3,907204290308,2021-12-05 07:57:04 UTC,89.130.89.27,es,community of madrid,alcobendas,Dalvik/2.1.0 (Linux; U; Android 7.1.1; H3311 B...,Other,Other
4,907204290308,2021-12-05 07:57:01 UTC,89.130.89.27,es,community of madrid,alcobendas,com.google.android.gms/214218020 (Linux; U; An...,Other,Other


In [29]:
def activ_dispositives():
    for i in files_list:
        if i == 'Actividad de registro de accesos':
            local = f"{path_string}/{i}/Dispositivos_ una lista con los dispositivos (por_.csv"
    df = pd.read_csv(local)
    return df

In [30]:
activ_dispositives = activ_dispositives()

In [31]:
activ_dispositives.drop(columns=['User Given Name'], inplace=True)

In [32]:
activ_dispositives.head()

,Device Type,Brand Name,Marketing Name,OS,OS Version,Device Model,Device Last Location,Gaia ID
0,MOBILE,Sony,Xperia L2,Android,7.1.1,H3311,Country ISO: ES\nLast Activity Time: 2021-12-0...,907204290308
1,PC,NaN,NaN,Windows,10.0,NaN,Country ISO: ES\nLast Activity Time: 2021-12-0...,907204290308


In [ ]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### YouTube y YouTube Music / (csv) [***COMPLETED***]

In [33]:
def youtube_music():
    for i in files_list:
        if i == 'YouTube y YouTube Music':
            local = f"{path_string}/{i}/canciones-biblioteca-musica/canciones-biblioteca-musica.csv"
    df = pd.read_csv(local)
    return df

In [34]:
youtube_music = youtube_music()

In [35]:
for i in range(len(youtube_music['URL de la canción'])):
    if '//' in youtube_music['URL de la canción'][i]:
        youtube_music['URL de la canción'][i] = youtube_music['URL de la canción'][i].split('//')[1]

for i in range(len(youtube_music['URL de la canción'])):
    if '/' in youtube_music['URL de la canción']:
        youtube_music['URL de la canción'][i] = youtube_music['URL de la canción'][i].split('/')[0]

In [36]:
youtube_music.head()

,URL de la canción,Título de la canción,Título del álbum,Nombre de los artistas
0,music.youtube.com/watch?v=6xzN8Nt0Pok,I Wanna Dance with Somebody (Who Loves Me),Whitney,Whitney Houston
1,music.youtube.com/watch?v=ojx78Bawl1M,Tuxedo Junction,The Age Of Swing,The BBC Big Band Orchestra
2,music.youtube.com/watch?v=cnRB2CgUpSw,Whatever It Takes,Evolve,Imagine Dragons


In [ ]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### Chrome / (html, json) [***COMPLETED***]

In [32]:
#json
def chrome_browser_history():
    for i in files_list:
        if i == 'Chrome':
            local = f"{path_string}/{i}/BrowserHistory.json"
    try:
        f = open(local)
        data = json.load(f)
        json_normalize(data)
        df = pd.DataFrame(data['Browser History'])
        return df
    except NameError:
        return 

In [33]:
chrome_browser_history = chrome_browser_history()

In [34]:
'''for i in range(len(chrome_browser_history.url)):
    if '//' in chrome_browser_history.url[i]:
        chrome_browser_history.url[i] = chrome_browser_history.url[i].split('//')[1]

for i in range(len(chrome_browser_history.url)):
    if '/' in chrome_browser_history.url[i]:
        chrome_browser_history.url[i] = chrome_browser_history.url[i].split('/')[0]'''

"for i in range(len(chrome_browser_history.url)):\n    if '//' in chrome_browser_history.url[i]:\n        chrome_browser_history.url[i] = chrome_browser_history.url[i].split('//')[1]\n\nfor i in range(len(chrome_browser_history.url)):\n    if '/' in chrome_browser_history.url[i]:\n        chrome_browser_history.url[i] = chrome_browser_history.url[i].split('/')[0]"

In [35]:
chrome_browser_history.head(2)

,page_transition,title,url,client_id,time_usec
0,LINK,Horarios de autobús 151: Madrid (Plaza Castill...,https://www.redtransporte.com/madrid/autobuses...,x6cQo08w9vTcM7buu4Mugg==,1638781091719312
1,LINK,Noticias sobre transporte público de Madrid,https://www.redtransporte.com/madrid/noticias....,x6cQo08w9vTcM7buu4Mugg==,1638781063045635


In [36]:
#html
def chrome_bookmarks():
    urls = []
    for i in files_list:
        if i == 'Chrome':
            local = f"{path_string}/{i}/Bookmarks.html"
            with open(local, 'r') as file:
                beautifulSoupText = BeautifulSoup(file.read(), 'html.parser')
                for tag in beautifulSoupText.findAll('a'):
                    urls.append(tag)
    return urls

In [37]:
chrome_bookmarks = chrome_bookmarks()

In [38]:
dicc = {'url' : [],
       'date': []
       }

In [39]:
for i in chrome_bookmarks:
    dicc["url"].append(i.get('href'))
    dicc["date"].append(i.get('add_date'))

In [40]:
df_chrome_bookmarks = pd.DataFrame(dicc)

In [41]:
'''for i in range(len(df_chrome_bookmarks.url)):
    if '//' in df_chrome_bookmarks.url[i]:
        df_chrome_bookmarks.url[i] = df_chrome_bookmarks.url[i].split('//')[1]

for i in range(len(df_chrome_bookmarks.url)):
    if '/' in df_chrome_bookmarks.url[i]:
        df_chrome_bookmarks.url[i] = df_chrome_bookmarks.url[i].split('/')[0]'''

"for i in range(len(df_chrome_bookmarks.url)):\n    if '//' in df_chrome_bookmarks.url[i]:\n        df_chrome_bookmarks.url[i] = df_chrome_bookmarks.url[i].split('//')[1]\n\nfor i in range(len(df_chrome_bookmarks.url)):\n    if '/' in df_chrome_bookmarks.url[i]:\n        df_chrome_bookmarks.url[i] = df_chrome_bookmarks.url[i].split('/')[0]"

In [42]:
df_chrome_bookmarks.head(2)

,url,date
0,https://www.institutoeuropeo.net/home/,13212144642644231
1,https://learndigital.withgoogle.com/activate/p...,13213188272419415


In [ ]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### Google Play Store / (json) [***COMPLETED***]

In [48]:
def google_pl_sto_dev():
    for i in files_list:
        if i == 'Google Play Store':
            local = f"{path_string}/{i}/Devices.json"
    try:
        f = open(local)
        data = json.load(f)
        temp = json_normalize(data)
        return temp
    except NameError:
        return 

In [49]:
google_pl_sto_dev = google_pl_sto_dev()

In [50]:
google_pl_sto_dev.rename(columns={'device.mostRecentData.carrierName':'CarrierName',
                                  'device.mostRecentData.playStoreClientVersion':'ClientVersion',
                                  'device.mostRecentData.manufacturer':'Manufacturer',
                                  'device.mostRecentData.modelName':'Model',
                                  'device.mostRecentData.deviceName':'DeviceName',
                                  'device.mostRecentData.productName':'ProductName',
                                  'device.mostRecentData.retailBrand':'RetailBrand',
                                 'device.mostRecentData.totalMemoryBytes':'Memory',
                                 'device.mostRecentData.deviceIpCountry':'Country',
                                 'device.mostRecentData.userLocale':'LocalUser',
                                  'device.mostRecentData.nativePlatform':'Platform',
                                 'device.mostRecentData.buildFingerprint':'Fingerprint',
                                 'device.mostRecentData.androidSdkVersion':'AndroidVersion',
                                 'device.deviceRegistrationTime':'ActiveSince',
                                 'device.userAddedOnDeviceTime':'TimeLapse',
                                 'device.lastTimeDeviceActive':'LastTimeActive'}, inplace=True)

In [51]:
google_pl_sto_dev.drop(columns=['Platform'], inplace=True)

In [52]:
google_pl_sto_dev.head()

,CarrierName,ClientVersion,Manufacturer,Model,DeviceName,ProductName,RetailBrand,Memory,Country,LocalUser,Fingerprint,AndroidVersion,ActiveSince,TimeLapse,LastTimeActive
0,Orange,82821010,Sony,H3311,H3311,H3311,Sony,3042852864,ES,es_ES,Sony/H3311/H3311:7.1.1/49.0.A.6.107/1689623269...,25,2021-03-20T15:49:29.415Z,2021-03-20T15:52:30.583Z,2021-12-04T22:30:38.817567Z
1,Orange,82442310,HMD Global,TA-1032,NE1,TA-1032_00WW,Nokia,1955610624,ES,es_ES,Nokia/TA-1032_00WW/NE1:9/PPR1.180610.011/00WW_...,28,2017-11-10T17:54:07.398Z,2017-12-19T23:49:13.181Z,2021-03-20T15:43:57.299247Z


In [53]:
def google_pl_sto_installs():
    for i in files_list:
        if i == 'Google Play Store':
            local = f"{path_string}/{i}/Installs.json"
    try:
        f = open(local)
        data = json.load(f)
        temp = json_normalize(data)
        return temp
    except NameError:
        return 

In [54]:
google_pl_sto_installs = google_pl_sto_installs()

In [55]:
google_pl_sto_installs.rename(columns={'install.doc.documentType':'From',
                                  'install.doc.title':'Title',
                                  'install.firstInstallationTime':'InstallationTime',
                                  'install.deviceAttribute.model':'Model',
                                  'install.deviceAttribute.carrier':'CarrierName',
                                  'install.deviceAttribute.manufacturer':'Manufacturer',
                                  'install.lastUpdateTime':'UpdateTime'}, inplace=True)

In [56]:
google_pl_sto_installs.drop(columns=['install.deviceAttribute.deviceDisplayName'], axis=1, inplace=True)

In [57]:
google_pl_sto_installs.head(3)

,From,Title,InstallationTime,Model,CarrierName,Manufacturer,UpdateTime
0,Android Apps,Meep - Rutas personalizadas,2021-05-21T15:37:51.377431Z,H3311,Orange,Sony,2021-11-29T10:23:41.762880Z
1,Android Apps,Google Chrome: rápido y seguro,2021-03-20T15:49:34.794045Z,H3311,Orange,Sony,2021-11-20T04:35:45.058178Z
2,Android Apps,Google Chrome: rápido y seguro,2017-11-15T17:32:15.465Z,TA-1032,Orange,HMD Global,2021-03-16T11:59:23.052276Z


In [ ]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### Google Shopping (txt) [***COMPLETED***]

In [183]:
def google_shop_address():
    for i in files_list:
        if i == 'Google Shopping':
            local = f"{path_string}/{i}/Addresses/Addresses.txt"
    try:
        myfile = open(local, "rt")
        contents = myfile.read()
        temp = re.split(',|\n', contents)
        myfile.close()
        return temp
    except NameError:
        return 

In [184]:
google_shop_address = google_shop_address()[2:]

In [185]:
addr_dicc = {'name': google_shop_address[0],
            'phone': google_shop_address[1],
            'address': google_shop_address[2]}

In [186]:
df_google_shop_address = pd.DataFrame(addr_dicc, index=(0,))

In [187]:
df_google_shop_address.head()

,name,phone,address
0,José Ramón Díaz,+34 615 68 63 48,Calle del Pintor Murillo Bajo A Alcobendas Mad...


In [ ]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### Maps (json) [***COMPLETED***]

In [43]:
def maps_favorites():
    for i in files_list:
        if i == 'Maps':
            local = f"{path_string}/{i}/Mis sitios etiquetados/Sitios etiquetados.json"
    try:
        f = open(local)
        data = json.load(f)
        return data
    except NameError:
        return 

In [44]:
maps_favorites = maps_favorites()

In [45]:
maps_favorites_dicc = {"coordinates": [], 
                        'name': [], 
                        'address': []}
for i in range(len(maps_favorites)):
    maps_favorites_dicc["coordinates"].append(maps_favorites['features'][i]['geometry']['coordinates'])
    maps_favorites_dicc["name"].append(maps_favorites["features"][i]['properties']['name'])
    maps_favorites_dicc["address"].append(maps_favorites["features"][i]['properties']["address"]) 

In [46]:
df_maps_favorites = pd.DataFrame(maps_favorites_dicc)

In [47]:
type(df_maps_favorites['coordinates'])

pandas.core.series.Series

In [48]:
for i in range(len(chrome_browser_history.url)):
    if '//' in chrome_browser_history.url[i]:
        chrome_browser_history.url[i] = chrome_browser_history.url[i].split('//')[1]

<ipython-input-48-9439837d9b9d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chrome_browser_history.url[i] = chrome_browser_history.url[i].split('//')[1]


In [49]:
'''split_data = df_maps_favorites.coordinates.str.split(', ')
df_maps_favorites['lat'] = split_data.apply(lambda x: x[0])
df_maps_favorites['long'] = split_data.apply(lambda x: x[1])'''

"split_data = df_maps_favorites.coordinates.str.split(', ')\ndf_maps_favorites['lat'] = split_data.apply(lambda x: x[0])\ndf_maps_favorites['long'] = split_data.apply(lambda x: x[1])"

In [50]:
df_maps_favorites

,coordinates,name,address
0,"[-3.6396805999999997, 40.5392152]",Casa,"Calle del Pintor Murillo, 4, 28100 Alcobendas,..."
1,"[-3.7046802, 40.4792542]",Trabajo,"Av. de Monforte de Lemos, Madrid, España"


In [51]:
def maps_own_sites():
    for i in files_list:
        if i == "Maps (Tus sitios)":
            local = f"{path_string}/{i}/Sitios guardados.json"
    try:
        f = open(local)
        data = json.load(f)
        return data
    except NameError:
        return

In [52]:
maps_own_sites = maps_own_sites()

In [53]:
maps_own_sites_dicc = {"coordinates": [],
                       'address': [],
                        'name': [],
                       'title': [],
                       'date': []
                        }
for i in range(len(maps_own_sites_dicc)):
    maps_own_sites_dicc["coordinates"].append(maps_own_sites['features'][i]['geometry']['coordinates'])
    maps_own_sites_dicc["address"].append(maps_own_sites["features"][i]['properties']['Location']["Address"])
    maps_own_sites_dicc["name"].append(maps_own_sites["features"][i]['properties']['Location']['Business Name'])
    maps_own_sites_dicc["title"].append(maps_own_sites["features"][i]['properties']['Title'])
    maps_own_sites_dicc["date"].append(maps_own_sites["features"][i]['properties']['Updated'])

In [54]:
df_maps_own_sites = pd.DataFrame(maps_own_sites_dicc)

In [55]:
df_maps_own_sites.drop(columns=['title'], axis=1, inplace=True)

In [56]:
df_maps_own_sites

,coordinates,address,name,date
0,"[10.7349734, 59.9179351]","St. Olavs Gate 31, 0166 Oslo, Noruega",Thon Hotel Europa,2019-09-26T10:57:03Z
1,"[10.7339016, 59.9185713]","Holbergs gate 30, 0166 Oslo, Noruega","Radisson Blu Scandinavia Hotel, Oslo",2019-09-26T10:56:31Z
2,"[10.7367862, 59.9177177]","St. Olavs Gate 26, 0166 Oslo, Noruega",Smarthotel Oslo,2019-09-26T10:56:04Z
3,"[10.748712, 59.9106631]","Fred Olsens gate 11, 0152 Oslo, Noruega",Comfort Hotel Xpress Central Station,2019-09-26T10:55:37Z
4,"[10.7165944, 59.9421593]","Forskningsparken Aksjeselskap, Gaustadalléen 2...",Forskningsparken - Oslo Science Park,2019-09-26T10:54:41Z


In [ ]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### Gmail (mbox) [***COMPLETED***]

In [155]:
for i in files_list:
        if i == "Correo":
            local = f"{path_string}/{i}/Todo el correo, incluido Spam y Papelera.mbox"

In [156]:
email_list = []

In [157]:
### define all the things ###
def quarters(date):
# a function to print which quarter an email was in

    winter = [1,2,3] #january through march
    spring = [4,5,6] #april through june
    summer = [7,8] #july through september
    fall = [9,10,11,12] #october through december
    if date.month in winter:
        return "WN"
    elif date.month in spring:
        return "SPR"
    elif date.month in summer:
        return "SU"
    elif date.month in fall:
        return "FA"

def d_o_w(date):
#a function that returns the day on which a call was recorded
    
    days_of_week = {0:"Mon", 1:"Tues", 2:"Wed", 3:"Thurs", 4:"Fri", 5:"Sat", 6:"Sun"}
    temp = date.reset_index(drop=True)
    
    #subtract 12 hours as a buffer for late call logs throughout the night, since calls
    #can come in at up to 2 am, but call logs can be submitted at any time in the next day
    
    temp_day = temp["Date"] - pd.Timedelta(hours=12) 
    day = []
    for i in temp_day:
        day.append(days_of_week.get(i.dayofweek))
    return day

""" 
the following two functions are courtesy
of stack overflow user R. Fabbiri

http://stackoverflow.com/questions/7166922/extracting
-the-body-of-an-email-from-mbox-file-decoding-it-to
-plain-text-regard
"""
def getcharsets(msg):
    charsets = set({})
    for c in msg.get_charsets():
        if c is not None:
            charsets.update([c])
    return charsets

def getBody(msg):
    while msg.is_multipart():
        msg=msg.get_payload()[0]
    t=msg.get_payload(decode=True)
    for charset in getcharsets(msg):
        t=t.decode(charset)
    t = lxml.html.fromstring(t.lower()).text_content()
    t = unicodedata.normalize("NFKD", t)
    t = re.sub(r'[\t\n\r]', ' ', t)
    return t

In [110]:
full_mail = mailbox.mbox(local)
print(type(full_mail))
print(full_mail[0].keys())
print("This mailbox has " + str(len(full_mail)) + " messages.")

<class 'mailbox.mbox'>
['X-GM-THRID', 'X-Gmail-Labels', 'Delivered-To', 'Received', 'X-Google-Smtp-Source', 'X-Received', 'ARC-Seal', 'ARC-Message-Signature', 'ARC-Authentication-Results', 'Return-Path', 'Received', 'Received-SPF', 'Authentication-Results', 'Received', 'From', 'To', 'Subject', 'Content-Type', 'X-Mailer', 'MIME-Version', 'Message-ID', 'Return-Path', 'Date']
This mailbox has 3408 messages.


In [111]:
n = random.randrange(0, len(full_mail))
print("AN EXAMPLE OF THE SUBJECT: " + full_mail[n]["Subject"])
print("\nAN EXAMPLE OF THE SENDER: " + full_mail[n]["From"])
print("\nAN EXAMPLE OF THE RECEPIENTS: " + full_mail[n]["To"])
print("\nAN EXAMPLE OF THE BODY: " + str(full_mail[n].get_payload()[0]))

AN EXAMPLE OF THE SUBJECT: =?UTF-8?Q?Su_candidatura_en_la_oferta_MONITORES_PARA_TALLERES_CIENT=C3=8DFICOS_ha_sido_remitida_a_la_empresa?=

AN EXAMPLE OF THE SENDER: BOLSA DE EMPLEO DEL AYUNTAMIENTO DE ALCOBENDAS
	<bolsaempleo@aytoalcobendas.org>

AN EXAMPLE OF THE RECEPIENTS: <jrdiazordaz03@gmail.com>

AN EXAMPLE OF THE BODY: <


In [124]:
print("AN EXAMPLE OF THE DATE: " + full_mail[n]["Date"])

AN EXAMPLE OF THE DATE: Mon, 4 Oct 2021 14:07:58 +0200


In [158]:
fields = ['Date', 'From', 'To', 'Subject','X-Gmail-Labels', 'X-GM-THRID']
allMessages = []

for message in tqdm(full_mail.itervalues(), desc='Processing'):
    dmessage = dict(message.items())
    allMessages.append({field:dmessage[field] if field in dmessage.keys() else '' for field in fields})

mail = pd.DataFrame(allMessages)

Processing: 3408it [01:27, 39.13it/s] 


In [159]:
mail['Date'] = mail['Date'].str.replace("\+.*", '', regex=True)

In [160]:
mail['Date'] = mail['Date'].str.replace(",", '')
mail['FinalDate'] = mail['Date'].str.findall(r'\d{1,2}\s\w{3}\s\d{4}')

In [161]:
def extract_element(x):
    return x[0]

In [162]:
mail['FinalDate'] = mail['FinalDate'].apply(extract_element)

In [163]:
mail['Time'] = mail['Date']

In [164]:
for i in range(len(mail['Date'])):
    mail['Time'][i] = mail['Date'][i][-9:]

In [165]:
mail['date'] = pd.to_datetime(mail['FinalDate'], utc=False)

In [166]:
mail.drop(columns=['Date', 'FinalDate'], inplace=True)

In [167]:
for i in range(len(mail['From'])):
    if '<' in mail['From'][i] or '>' in mail['From'][i]:
        mail['From'][i] = mail['From'][i].replace('<', '')
        mail['From'][i] = mail['From'][i].replace('>', '')

<ipython-input-167-7777946a0e48>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mail['From'][i] = mail['From'][i].replace('<', '')
<ipython-input-167-7777946a0e48>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mail['From'][i] = mail['From'][i].replace('>', '')


TypeError: argument of type 'Header' is not iterable

In [168]:
for i in range(len(mail['To'])):
    mail['To'][i] = mail['To'][i].replace('<', '')
    mail['To'][i] = mail['To'][i].replace('>', '')

<ipython-input-168-b3e438c4ea7f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mail['To'][i] = mail['To'][i].replace('<', '')
<ipython-input-168-b3e438c4ea7f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mail['To'][i] = mail['To'][i].replace('>', '')


AttributeError: 'Header' object has no attribute 'replace'

In [169]:
bodies = []

for i in tqdm(range(len(full_mail)), desc='Processing'):
    try:
        t = getBody(full_mail[i])
        t = lxml.html.fromstring(t.lower()).text_content()
        t = unicodedata.normalize("NFKD", t)
        bodies.append(t)
    except:
        bodies.append("")
# remove pesky replies and signatures
bodies = [re.sub("on \d\d/\d\d/\d\d.+","",bodies[n]) for n in range(len(bodies))]
bodies = [re.sub("--.+","",bodies[n]) for n in range(len(bodies))]
bodies = [re.sub("_+", "",bodies[n]) for n in range(len(bodies))]

mail["body"] = bodies
mail["quarter"]=[quarters(i) for i in mail["date"]]


mail.head(2)

Processing: 100%|██████████| 3408/3408 [01:31<00:00, 37.14it/s] 


,From,To,Subject,labels,thread,Time,date,body,quarter
0,provomad@cruzroja.es,jrdiazordaz03@gmail.com,III Jornadas Creando igualdad: di���logos con ...,"Recibidos,Abierto",1718080706845359492,00:57:23,2021-12-03,estimado jose ramon: los próximos días 14...,FA
1,supportlearning@ifrc.org,jrdiazordaz03@gmail.com,=?utf-8?B?SGVsbG8gSm9zw6kgUmFtw7NuIC0gTGVhcm5p...,"Recibidos,Abierto",1718143568455302745,:52 -0800,2021-12-03,*|mc:subject|* p{ margi...,FA


In [171]:
mail = mail.rename(columns = {"X-GM-THRID" : "thread","X-Gmail-Labels" : "labels", 'From':'from', 'To':'to'})

In [172]:
mail.head(1)

,from,to,Subject,labels,thread,Time,date,body,quarter
0,provomad@cruzroja.es,jrdiazordaz03@gmail.com,III Jornadas Creando igualdad: di���logos con ...,"Recibidos,Abierto",1718080706845359492,00:57:23,2021-12-03,estimado jose ramon: los próximos días 14...,FA


In [ ]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### Location History (json) --> no tengo datos míos, es muy difícil la estructura

In [ ]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### Google Fotos (json) [***COMPLETED***]

In [83]:
def folder_selector():
    for i in files_list:
        if i == "Google Fotos":
            local = f"{path_string}/{i}/"
            
    with os.scandir(local) as ficheros:
        ficheros = [fichero.name for fichero in ficheros if fichero.is_file() == False]
    return ficheros

def folder_iterator(*args, **kwargs):
    for i in files_list:
        if i == "Google Fotos":
            local = f"{path_string}/{i}/"
    directory = []
    for i in folder_selector():
        path = f"{local}{i}"
        directory.append(path)
    return directory

def csv_scanner(*args, **kwargs):
    csvs = []
    for j in folder_iterator():
        os.chdir(j)
        temp_ = glob.glob("*.json")
        csvs.append(temp_)
    #g1 = [i.replace("'", '') for i in csvs]
    return csvs

def merge_JsonFiles(*args, **kwargs):
    result = []
    for f1 in csv_scanner():
        for j in f1:
            f = open(j)
            data = json.load(f)
            result.append(data)
    return result

def pics_info_extractor(*args, **kwargs):
    result = merge_JsonFiles()
    title_ = [i["title"] for i in result]
    description_ = [i["description"] for i in result]
    imageViews_ = [i["imageViews"] for i in result]
    timestamp_ = [i["photoTakenTime"]['timestamp'] for i in result]
    dicct_ = {'title':title_,'description':description_,'imageViews':imageViews_,'timestamp':timestamp_}
    df = pd.DataFrame(dicct_, columns=('title','description','imageViews','timestamp'))
    return df

In [84]:
pics_info_extractor = pics_info_extractor()

In [85]:
pics_info_extractor

,title,description,imageViews,timestamp
0,IMG_20200709_121740.jpg,,7,1594289860


In [ ]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### Drive (varios)

In [86]:
def drive_scanner():
    for i in files_list:
        if i == "Drive":
            local = f"{path_string}/{i}/"
            
    pdf = []
    mp4 = []
    xls = []
    pptx = []
    html = []
    kmz = []
    flv = []
    mp3 = []
    xlsx = []
    jpg = []
    docx = []
    txt = []

    for root, dirs, files in os.walk(local):
    # select file name
        for file in files:
        # check the extension of files
            if file.endswith('.pdf'):
                pdf.append(os.path.join(file))
            if file.endswith('.mp4'):
                mp4.append(os.path.join(file))
            if file.endswith('.xls'):
                xls.append(os.path.join(file))
            if file.endswith('.pptx'):
                pptx.append(os.path.join(file))
            if file.endswith('.html'):
                html.append(os.path.join(file))
            if file.endswith('.kmz'):
                kmz.append(os.path.join(file))
            if file.endswith('.flv'):
                flv.append(os.path.join(file))
            if file.endswith('.mp3'):
                mp3.append(os.path.join(file))
            if file.endswith('.xlsx'):
                mp3.append(os.path.join(file))
            if file.endswith('.jpg'):
                mp3.append(os.path.join(file))
            if file.endswith('.docx'):
                mp3.append(os.path.join(file))
            if file.endswith('.txt'):
                mp3.append(os.path.join(file))
    
    dictionary = {'pdf':pdf,'mp4':mp4,'xls':xls,'pptx':pptx,'html':html,'kmz':kmz,'flv':flv,
                  'mp3':mp3,'xlsx':xlsx,'jpg':jpg,'docx':docx,'txt':txt}
    
    serie = pd.Series(dictionary)
    
    df = pd.DataFrame(serie).T
    
    return df


In [158]:
def drive_counter():
    for i in files_list:
        if i == "Drive":
            local = f"{path_string}/{i}/"

    os.chdir(local)
    cnt = collections.Counter()
    for filename in glob.glob("*"):
        name, ext = os.path.splitext(filename)
        cnt[ext] += 1
    df = pd.DataFrame(cnt, index=(0,))
    return df

In [159]:
drive_counter = drive_counter()

In [152]:
drive_counter.head()

,.pdf,.docx,,.mp3,.txt,.doc,.zip,.xls,.xlsx,.JPG,.pptx,.mp4,.html,.kmz,.flv
0,6,2,18,9,1,1,1,1,5,1,3,5,1,1,1


In [87]:
drive_scanner = drive_scanner()

In [88]:
drive_scanner

,pdf,mp4,xls,pptx,html,kmz,flv,mp3,xlsx,jpg,docx,txt
0,"[20201013 El Plan Responde en datos.pdf, CV_EN...","[KIZOA-Movie-Maker-29yinzuv.mp4, MOV_0016.mp4,...","[Escape Rooms.xls, H2020-PHC-2014_CNECT_Annexe...",[INCLIVA_conv. 2020 SC1 H2020_HE_03.09.2019_ v...,"[Modulo5y6.html, KBDOC-00001.html, KBDOC-00002...",[Participación española en Infraestructuras de...,[The uses of Tai Chi Chuan.flv],"[Abstract.docx, Can_t Hold Us corto.mp3, Can_t...",[],[],[],[]


In [89]:
drive_scanner['pdf']

0    [20201013 El Plan Responde en datos.pdf, CV_EN...
Name: pdf, dtype: object

In [ ]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# Merge

Tables:
 - personal_info
 - activity
 - dispositives
 - music
 - history
 - favorites
 - locations
 - photos
 - drive
 - gmail

### 1 - personal_info

In [91]:
li_final.head()

,Google Account ID,Created on,IP,Language,e-mail,recovery mail,phone
0,907204290308,2017-07-20 08,213.4.207.110,es,JRDiazOrdaz03@gmail.com,IronhideX1@gmail.com,+34615686348 [ES]


In [93]:
df_profile_extr.head()

,name,email,gender
0,Jose Diaz,jrdiazordaz03@gmail.com,male


In [94]:
li_final.rename(columns={'email':'e-mail'}, inplace=True)

In [104]:
li_final['e-mail'] = li_final['e-mail'].str.strip().str.lower()

In [108]:
personal_info = li_final.merge(df_profile_extr, how='outer', on='e-mail')

In [208]:
personal_info.head()

,Google Account ID,Created on,IP,Language,e-mail,recovery mail,phone,name,gender
0,907204290308,2017-07-20 08,213.4.207.110,es,jrdiazordaz03@gmail.com,IronhideX1@gmail.com,+34615686348 [ES],Jose Diaz,male


In [112]:
personal_info.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/personal_info.csv', index=False)

### 2 - activity

In [209]:
tr.head()

,ip,activity,date
1,2.136.118.161,Login,2021-12-06 10:27:15 UTC
2,85.48.184.103,Login,2021-12-06 10:27:11 UTC
3,89.130.89.27,Login,2021-12-05 14:39:14 UTC
4,85.48.184.103,Login,2021-12-05 14:39:07 UTC
5,85.48.184.103,Login,2021-12-05 14:39:06 UTC


In [114]:
tr.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/activity.csv', index=False)

### 3 - dispositives

In [162]:
activ_services.head(3)

,Gaia ID,Activity Timestamp,IP Address,Activity Country,Activity Region,Activity City,User Agent String,Product Name,Sub-Product Name
0,907204290308,2021-12-05 07:57:48 UTC,89.130.89.27,es,community of madrid,alcobendas,Android-Gmail/62948716 (sw360dp; 320dpi) (H331...,Gmail,Other
1,907204290308,2021-12-05 07:57:47 UTC,89.130.89.27,es,community of madrid,alcobendas,Android-Gmail/62948716 (sw360dp; 320dpi) (H331...,Gmail,Other
2,907204290308,2021-12-05 07:57:17 UTC,89.130.89.27,es,community of madrid,alcobendas,Android-Gmail/62948716 (sw360dp; 320dpi) (H331...,Gmail,Other


In [223]:
activ_services.shape

(60301, 9)

In [163]:
activ_dispositives.head(3)

,Device Type,Brand Name,Marketing Name,OS,OS Version,Device Model,Device Last Location,Gaia ID
0,MOBILE,Sony,Xperia L2,Android,7.1.1,H3311,Country ISO: ES\nLast Activity Time: 2021-12-0...,907204290308
1,PC,NaN,NaN,Windows,10.0,NaN,Country ISO: ES\nLast Activity Time: 2021-12-0...,907204290308


In [215]:
dispositives = activ_services.merge(activ_dispositives, how='inner', on='Gaia ID')

In [216]:
google_pl_sto_installs.head()

,From,Title,InstallationTime,Model,CarrierName,Manufacturer,UpdateTime
0,Android Apps,Meep - Rutas personalizadas,2021-05-21T15:37:51.377431Z,H3311,Orange,Sony,2021-11-29T10:23:41.762880Z
1,Android Apps,Google Chrome: rápido y seguro,2021-03-20T15:49:34.794045Z,H3311,Orange,Sony,2021-11-20T04:35:45.058178Z
2,Android Apps,Google Chrome: rápido y seguro,2017-11-15T17:32:15.465Z,TA-1032,Orange,HMD Global,2021-03-16T11:59:23.052276Z
3,Android Apps,Google Chrome: rápido y seguro,2020-02-03T16:57:00.724Z,,,,2020-03-19T23:20:52.820825Z
4,Android Apps,Grabadora de Voz Fácil,2021-11-05T09:06:44.355204Z,H3311,Orange,Sony,2021-11-05T09:06:44.355204Z


In [217]:
dispositives.rename(columns={'Device Model':'Model'}, inplace=True)

In [218]:
dispositives.head(3)

,Gaia ID,Activity Timestamp,IP Address,Activity Country,Activity Region,Activity City,User Agent String,Product Name,Sub-Product Name,Device Type,Brand Name,Marketing Name,OS,OS Version,Model,Device Last Location
0,907204290308,2021-12-05 07:57:48 UTC,89.130.89.27,es,community of madrid,alcobendas,Android-Gmail/62948716 (sw360dp; 320dpi) (H331...,Gmail,Other,MOBILE,Sony,Xperia L2,Android,7.1.1,H3311,Country ISO: ES\nLast Activity Time: 2021-12-0...
1,907204290308,2021-12-05 07:57:48 UTC,89.130.89.27,es,community of madrid,alcobendas,Android-Gmail/62948716 (sw360dp; 320dpi) (H331...,Gmail,Other,PC,NaN,NaN,Windows,10.0,NaN,Country ISO: ES\nLast Activity Time: 2021-12-0...
2,907204290308,2021-12-05 07:57:47 UTC,89.130.89.27,es,community of madrid,alcobendas,Android-Gmail/62948716 (sw360dp; 320dpi) (H331...,Gmail,Other,MOBILE,Sony,Xperia L2,Android,7.1.1,H3311,Country ISO: ES\nLast Activity Time: 2021-12-0...


In [219]:
dispositives1 = dispositives.merge(google_pl_sto_installs, how='inner', on='Model')

In [1]:
dispositives1.keys

NameError: name 'dispositives1' is not defined

In [221]:
dispositives1.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/dispositives.csv', index=False)

In [5]:
dispositives1 = pd.read_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/dispositives.csv')

In [8]:
dispositives1.head(1)

,Gaia ID,Activity Timestamp,IP Address,Activity Country,Activity Region,Activity City,User Agent String,Product Name,Sub-Product Name,Device Type,...,OS,OS Version,Model,Device Last Location,From,Title,InstallationTime,CarrierName,Manufacturer,UpdateTime
0,907204290308,2021-12-05 07:57:48 UTC,89.130.89.27,es,community of madrid,alcobendas,Android-Gmail/62948716 (sw360dp; 320dpi) (H331...,Gmail,Other,MOBILE,...,Android,7.1.1,H3311,Country ISO: ES\nLast Activity Time: 2021-12-0...,Android Apps,Meep - Rutas personalizadas,2021-05-21T15:37:51.377431Z,Orange,Sony,2021-11-29T10:23:41.762880Z


In [222]:
dispositives1.shape

(1989933, 22)

### 4 - music

In [161]:
youtube_music.head()

,URL de la canción,Título de la canción,Título del álbum,Nombre de los artistas
0,music.youtube.com/watch?v=6xzN8Nt0Pok,I Wanna Dance with Somebody (Who Loves Me),Whitney,Whitney Houston
1,music.youtube.com/watch?v=ojx78Bawl1M,Tuxedo Junction,The Age Of Swing,The BBC Big Band Orchestra
2,music.youtube.com/watch?v=cnRB2CgUpSw,Whatever It Takes,Evolve,Imagine Dragons


In [116]:
youtube_music.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/music.csv', index=False)

### 5 - history

In [238]:
chrome_browser_history.head(3)

,page_transition,title,url,client_id,time_usec
0,LINK,Horarios de autobús 151: Madrid (Plaza Castill...,https://www.redtransporte.com/madrid/autobuses...,x6cQo08w9vTcM7buu4Mugg==,1638781091719312
1,LINK,Noticias sobre transporte público de Madrid,https://www.redtransporte.com/madrid/noticias....,x6cQo08w9vTcM7buu4Mugg==,1638781063045635
2,LINK,Horarios de autobús 151: Madrid (Plaza Castill...,https://www.redtransporte.com/madrid/autobuses...,x6cQo08w9vTcM7buu4Mugg==,1638781050258255


In [239]:
chrome_browser_history.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/history.csv', index=False)

### 6 - favorites

In [236]:
df_chrome_bookmarks.head(3)

,url,date
0,https://www.institutoeuropeo.net/home/,13212144642644231
1,https://learndigital.withgoogle.com/activate/p...,13213188272419415
2,https://www.open.edu/openlearncreate/course/vi...,13215605874184222


In [237]:
df_chrome_bookmarks.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/favorites.csv', index=False)

### 7 - locations

In [57]:
df_maps_favorites.head()

,coordinates,name,address
0,"[-3.6396805999999997, 40.5392152]",Casa,"Calle del Pintor Murillo, 4, 28100 Alcobendas,..."
1,"[-3.7046802, 40.4792542]",Trabajo,"Av. de Monforte de Lemos, Madrid, España"


In [58]:
df_maps_own_sites.head()

,coordinates,address,name,date
0,"[10.7349734, 59.9179351]","St. Olavs Gate 31, 0166 Oslo, Noruega",Thon Hotel Europa,2019-09-26T10:57:03Z
1,"[10.7339016, 59.9185713]","Holbergs gate 30, 0166 Oslo, Noruega","Radisson Blu Scandinavia Hotel, Oslo",2019-09-26T10:56:31Z
2,"[10.7367862, 59.9177177]","St. Olavs Gate 26, 0166 Oslo, Noruega",Smarthotel Oslo,2019-09-26T10:56:04Z
3,"[10.748712, 59.9106631]","Fred Olsens gate 11, 0152 Oslo, Noruega",Comfort Hotel Xpress Central Station,2019-09-26T10:55:37Z
4,"[10.7165944, 59.9421593]","Forskningsparken Aksjeselskap, Gaustadalléen 2...",Forskningsparken - Oslo Science Park,2019-09-26T10:54:41Z


In [59]:
temp = pd.concat([df_maps_favorites, df_maps_own_sites])

In [61]:
temp.head(10)

,coordinates,name,address,date
0,"[-3.6396805999999997, 40.5392152]",Casa,"Calle del Pintor Murillo, 4, 28100 Alcobendas,...",NaN
1,"[-3.7046802, 40.4792542]",Trabajo,"Av. de Monforte de Lemos, Madrid, España",NaN
0,"[10.7349734, 59.9179351]",Thon Hotel Europa,"St. Olavs Gate 31, 0166 Oslo, Noruega",2019-09-26T10:57:03Z
1,"[10.7339016, 59.9185713]","Radisson Blu Scandinavia Hotel, Oslo","Holbergs gate 30, 0166 Oslo, Noruega",2019-09-26T10:56:31Z
2,"[10.7367862, 59.9177177]",Smarthotel Oslo,"St. Olavs Gate 26, 0166 Oslo, Noruega",2019-09-26T10:56:04Z
3,"[10.748712, 59.9106631]",Comfort Hotel Xpress Central Station,"Fred Olsens gate 11, 0152 Oslo, Noruega",2019-09-26T10:55:37Z
4,"[10.7165944, 59.9421593]",Forskningsparken - Oslo Science Park,"Forskningsparken Aksjeselskap, Gaustadalléen 2...",2019-09-26T10:54:41Z


In [60]:
temp.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/locations.csv', index=False)

### 8 - photos

In [125]:
pics_info_extractor.head()

,title,description,imageViews,timestamp
0,IMG_20200709_121740.jpg,,7,1594289860


In [126]:
pics_info_extractor.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/photos.csv', index=False)

### 9 - drive

In [153]:
drive_counter.head()

,.pdf,.docx,,.mp3,.txt,.doc,.zip,.xls,.xlsx,.JPG,.pptx,.mp4,.html,.kmz,.flv
0,6,2,18,9,1,1,1,1,5,1,3,5,1,1,1


In [155]:
drive_counter.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/drive.csv', index=False)

### 10 - mail

In [177]:
mail.head(1)

,from,to,Subject,labels,thread,Time,date,body,quarter
0,provomad@cruzroja.es,jrdiazordaz03@gmail.com,III Jornadas Creando igualdad: di���logos con ...,"Recibidos,Abierto",1718080706845359492,00:57:23,2021-12-03,estimado jose ramon: los próximos días 14...,FA


In [178]:
mail.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/mail.csv', index=False)